In [1]:
from utils import *
import pandas as pd
import mne
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from sklearn.preprocessing import normalize, StandardScaler
plt.style.use('seaborn-whitegrid')

TYPE_DEF = {0:'Non-Stress', 1:'Neutral', 2: 'Stress'}
PSS = load('PSS')
type_count = load('type_count')

In [2]:
def get_freq(PSS):
    # peak at info
    temp = PSS.popitem()
    PSS[temp[0]] = temp[1]
    raw = temp[1]['raw']
    power,freq = mne.time_frequency.psd_welch(raw,n_fft=125, verbose=True)
    return freq


for name, info in PSS.items():
    raw = info['raw']
    # raw.filter(l_freq=1,h_freq=None, method='iir', iir_params={'order':3.0, 'ftype':'butter'}, verbose=False) # Slow drift
    # raw.notch_filter(freqs=[50])

freq = get_freq(PSS)
print(freq)


band_names = np.array(['Delta', 'Theta', 'Alpha', 'Beta', 'Gamma', 'Slow', 'Low_beta'])
filter_list = [[1,3],[4,7],[8,12],[13,30],[30,43], [4,13], [13,17]]
bands = []
for filt in filter_list:
    pt = np.argwhere((freq >= filt[0]) & (freq <= filt[1])).reshape(-1)
    bands.append(pt)
bands = np.array(bands)
print(bands)

def get_markers():
    sampling_rate = 125 #Hz
    # 15/60 = 0.25
    step_minutes = np.arange(0,5,0.25)
    print(f"{step_minutes=}")
    step_minutes = np.expand_dims(step_minutes * sampling_rate * 60, axis=1)
    markers = np.concatenate( [step_minutes, np.zeros( step_minutes.shape ), np.ones( step_minutes.shape ) ], axis=1  ).astype(np.int64)
    return markers
markers = get_markers()
# markers

Effective window size : 1.000 (s)
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34. 35.
 36. 37. 38. 39. 40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50. 51. 52. 53.
 54. 55. 56. 57. 58. 59. 60. 61. 62.]
[array([1, 2, 3]) array([4, 5, 6, 7]) array([ 8,  9, 10, 11, 12])
 array([13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29,
        30])
 array([30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43])
 array([ 4,  5,  6,  7,  8,  9, 10, 11, 12, 13])
 array([13, 14, 15, 16, 17])]
step_minutes=array([0.  , 0.25, 0.5 , 0.75, 1.  , 1.25, 1.5 , 1.75, 2.  , 2.25, 2.5 ,
       2.75, 3.  , 3.25, 3.5 , 3.75, 4.  , 4.25, 4.5 , 4.75])


/tmp/ipykernel_15770/3477651712.py:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  bands = np.array(bands)


In [3]:
for name,info in PSS.items():
    raw = info['raw']
    epochs = mne.Epochs(raw, markers, tmin=0, tmax=15, baseline=(0,2), verbose=False)
    feature_epochs = None
    for evoked in epochs.iter_evoked():
        feature = None
        slow, gamma = None, None
        a_f3, a_f4 = None, None
        a_t7, a_t8 = None, None
        b_f3, b_f4 = None, None
        b_t7, b_t8 = None, None
        for index, band in enumerate(bands):
            power,freq = mne.time_frequency.psd_welch(evoked,n_fft=125, verbose=False)
            power = power.squeeze()
            power = 10 * np.log10(power)
            data = power[::,band].mean(axis=1).reshape(1,-1)
            # for asym
            if(band_names[index] == 'Alpha'):
                a_f3 = data[:,raw.ch_names.index('F3')]
                a_f4 = data[:,raw.ch_names.index('F4')]
                # We use t3 as t7 and t4 as t8
                a_t7 = data[:,raw.ch_names.index('T3')]
                a_t8 = data[:,raw.ch_names.index('T4')]
            if(band_names[index] == 'Beta'):
                b_f3 = data[:,raw.ch_names.index('F3')]
                b_f4 = data[:,raw.ch_names.index('F4')]
                # We use t3 as t7 and t4 as t8
                b_t7 = data[:,raw.ch_names.index('T3')]
                b_t8 = data[:,raw.ch_names.index('T4')]

            ####### Mean for visualization #######
            data = data.reshape(1,-1)
            # print(data.shape)
            # for relative gamma
            if(band_names[index] == 'Slow'): slow = data
            if(band_names[index] == 'Gamma'): gamma = data

            if(type(feature) == type(None)): feature = data
            else: feature = np.concatenate([feature, data], axis=1)
        # print(feature.shape)
        # the eighth feature: relative gamma is slow/gamma
        relative_gamma = slow/gamma
        feature = np.concatenate([feature, relative_gamma], axis=1)
        # The asymetry
        alpha_frontal = ((a_f4 - a_f3) / (a_f4 + a_f3)).reshape(1,-1)
        feature = np.concatenate([feature, alpha_frontal], axis=1)
        # alpha_temporal
        alpha_temporal = ((a_t8 - a_t7) / (a_t8 + a_t7)).reshape(1,-1)
        feature = np.concatenate([feature, alpha_temporal], axis=1)
        # alpha_asymmetry
        alpha_asymmetry = alpha_frontal + alpha_temporal
        feature = np.concatenate([feature, alpha_asymmetry], axis=1)
        # beta_frontal
        beta_frontal = ((b_f4 - b_f3) / (b_f4 + b_f3)).reshape(1,-1)
        feature = np.concatenate([feature, beta_frontal], axis=1)
        # beta_temporal
        beta_temporal = ((b_t8 - b_t7) / (b_t8 + b_t7)).reshape(1,-1)
        feature = np.concatenate([feature, beta_temporal], axis=1)
        if(type(feature_epochs) == type(None)): feature_epochs = feature
        else: feature_epochs = np.concatenate( [feature_epochs, feature], axis=0 )
    info['feature'] = feature_epochs
print(f"{feature_epochs.shape=}")


feature_epochs.shape=(20, 133)


In [4]:
feature_names = []
for band_name in band_names:
    for ch_name in raw.ch_names[:-1]:
        feature_names.append(f"{ch_name}_{band_name}")
for ch_name in raw.ch_names[:-1]:
    feature_names.append(f"{ch_name}_relative_gamma")
feature_names.append("alpha_frontal")
feature_names.append("alpha_temporal")
feature_names.append("alpha_asymmetry")
feature_names.append("beta_frontal")
feature_names.append("beta_temporal")

headers = ["name","types","scores"] + feature_names
print(len(headers), len(feature_names), feature_epochs.shape)


136 133 (20, 133)


In [5]:
X_ori,y_ori = [], []
participants = []
scores = []
types = []
non_stress_count, stress_count = 0,0
for index,(name,info) in enumerate(PSS.items()):
    for feature in info['feature']:
        # Neutral
        # if(info['type'] == 1): continue
        # if(info['type'] == 1): continue
        # Non-Stress
        if(info['type'] == 0):
            non_stress_count = non_stress_count + 1
        #     # print(name, info['score'])
        #     y_ori.append(0)
        # Stress
        elif(info['type'] == 2):
            stress_count = stress_count + 1
        #     y_ori.append(1)
        X_ori.append(feature)
        participants.append(name)
        scores.append(info['score'])
        types.append(info['type'])
print(f"{non_stress_count=} {stress_count=}")
print(f"{np.array(X_ori).shape=}")
print(f"{len(participants)=}")
print(f"{len(scores)=}")
print(f"{len(types)=}")
# print(f"{np.array(y_ori).shape=}")


non_stress_count=320 stress_count=400
np.array(X_ori).shape=(1100, 133)
len(participants)=1100
len(scores)=1100
len(types)=1100


In [6]:
data = {'name':participants,
        'score':scores,
        'type':types}
for col,feature_name in zip(np.array(X_ori).T,feature_names):
    # print(feature_name,col.shape)
    data[feature_name] = list(col)

csv = pd.DataFrame(data)
csv.to_csv('export/15s_133features_all_baseline.csv')
csv[csv['type']!=1].to_csv('export/15s_133features_filter_baseline.csv')

In [7]:
def check_float(data):
    if(data.dtype == np.int64): data = data.astype(np.float64)
    return data

def StandardJa(data):
    data = check_float(data)
    for index, row in enumerate(data):
        mean = row.mean()
        std = row.std()
        row = (row - mean) / std
        data[index] = row
        # print(row)
    return data

X_ori_std = StandardJa(np.array(X_ori).copy().T).T

data = {'name':participants,
        'score':scores,
        'type':types}
for col,feature_name in zip(X_ori_std.T,feature_names):
    # print(feature_name,col.shape)
    data[feature_name] = list(col)

csv = pd.DataFrame(data)
csv.to_csv('export/15s_133features_all_std_baseline.csv')
csv[csv['type']!=1].to_csv('export/15s_133features_filter_std_baseline.csv')